In [2]:
import numpy as np
import pandas as pd

In [3]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

In [4]:
movies=movies.merge(credits,on="title")

In [5]:
movies=movies[["movie_id","title","overview","genres","keywords","cast","crew"]]

In [6]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [7]:
movies.dropna(inplace=True)

In [8]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    return L

In [10]:
movies["genres"]=movies["genres"].apply(convert)

In [11]:
movies["keywords"]=movies["keywords"].apply(convert)

In [12]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i["name"])
            counter+=1
        else:
            break
    return L

In [13]:
movies["cast"]=movies["cast"].apply(convert3)

In [14]:
def fetch_director(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if i["job"]=="Director":
            L.append(i["name"])
            break
    return L

In [15]:
movies["crew"]=movies["crew"].apply(fetch_director)

In [16]:
movies["overview"]=movies["overview"].apply(lambda x:x.split())

In [17]:
movies["genres"]=movies["genres"].apply(lambda x:[i.replace(" ","")for i in x])
movies["keywords"]=movies["keywords"].apply(lambda x:[i.replace(" ","")for i in x])
movies["cast"]=movies["cast"].apply(lambda x:[i.replace(" ","")for i in x])
movies["crew"]=movies["crew"].apply(lambda x:[i.replace(" ","")for i in x])

In [18]:
movies["tags"]=movies["overview"]+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]

In [19]:
new_df=movies[["movie_id","title","tags"]]

In [20]:
new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))

C:\Users\mayan\AppData\Local\Temp\ipykernel_27388\1804341217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))


In [21]:
new_df["tags"]=new_df["tags"].apply(lambda x:x.lower())

C:\Users\mayan\AppData\Local\Temp\ipykernel_27388\4024514323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x:x.lower())


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=8000,stop_words="english")

In [23]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word)for word in text.split()])

In [24]:
new_df["tags"]=new_df["tags"].apply(stem_words)

C:\Users\mayan\AppData\Local\Temp\ipykernel_27388\2989784099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(stem_words)


In [25]:
vectors=cv.fit_transform(new_df["tags"]).toarray()

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
similarities=cosine_similarity(vectors)

In [27]:
def recommend(movie):
    movie_index=new_df[new_df["title"]==movie].index[0]
    distances=similarities[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [28]:
recommend("Batman Begins")

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [29]:
import pickle
pickle.dump(new_df,open("movies.pkl","wb"))

In [30]:
pickle.dump(new_df.to_dict(),open("movies_dict.pkl","wb"))

In [31]:
pickle.dump(similarities,open("similarities.pkl","wb"))